# Propensity Matching Score

This notebook is intended to provide a basic routine of code to generate a Propensity Matching Score model.

## Motivation


A Propensity Matching Score model, tries to account for the causal relationship between a treatment and control variables on a set of covariates.
The treatment group being a dummy variavle; having or no having a condition. Matching is created to create an artificial control group and then to estimate the impact of treatment. The model assumes; unconfoundedness. Selection on treatment (or not) should be solely based on observable characteristics. Assuming there is no selection bias from unobserved characteristics. It is not possible to prove the validity of this unconfoundedness assumption.

### Steps followed: 

 - Estimate the propensity score. This is the propability (logistic regression) that an observation is treated or not. 
 - Perform matching. For each treated sample, identify an untreated sample with similar logit propensity score. The matching is 1-to-1 with replacement, using Nearest Neighbors technique.
 - Once matching is performed, we review the balance of the X variables to assess their balance. 
 - Estimate the impact of treatment with a T-test of proportions.


In [1]:
# Import all the necesarry libraries 

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.stats.proportion as sp
from statsmodels.stats.proportion import proportions_ztest
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [4]:
#Load the data set and make a copy
# The dataset used for this notebook belongs to MIT Sloan of Managment specifically to professor Sinan Aral

data = pd.read_csv('your data here')
df = data.copy()


## Pre-processing and data wrangling

In [5]:
# Pre-processing of the data set. 
df = df.drop(['delta2_shouts'], axis= 1)
df = df.dropna()

In [6]:
#Separate variables 
df= df[['adopter','age', 'male', 'tenure', 'good_country', 'friend_cnt', 'avg_friend_age',
       'avg_friend_male', 'friend_country_cnt', 'subscriber_friend_cnt',
       'songsListened', 'lovedTracks', 'posts', 'playlists', 'shouts']]

In [7]:
#Creating treatment and control variables 
df['treatment'] =np.where(df['subscriber_friend_cnt'] >=1, 1,0)
df_control = df[df.treatment==0]
df_treatment = df[df.treatment==1]

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 43827 entries, 2 to 107212
Data columns (total 16 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adopter                43827 non-null  int64  
 1   age                    43827 non-null  float64
 2   male                   43827 non-null  float64
 3   tenure                 43827 non-null  float64
 4   good_country           43827 non-null  float64
 5   friend_cnt             43827 non-null  float64
 6   avg_friend_age         43827 non-null  float64
 7   avg_friend_male        43827 non-null  float64
 8   friend_country_cnt     43827 non-null  float64
 9   subscriber_friend_cnt  43827 non-null  float64
 10  songsListened          43827 non-null  int64  
 11  lovedTracks            43827 non-null  int64  
 12  posts                  43827 non-null  int64  
 13  playlists              43827 non-null  int64  
 14  shouts                 43827 non-null  float64
 15  t

In [11]:
#Check the proportion of cases that are in the dependent variable (adopter) for the treatment and control group.
prop_total = df.treatment.value_counts()
prop_control = df_control.loc[df.adopter ==1, 'adopter'].count()
prop_treatment = df_treatment.loc[df.adopter ==1, 'adopter'].count()
print(prop_total, prop_control, prop_treatment)

0    34004
1     9823
Name: treatment, dtype: int64 1783 1744


In [12]:
#Run a T-test of proportions to check for independece among groups
stat, pval = proportions_ztest([1744, 1783], [9823, 34004])
print('{0:0.3f}'.format(pval))
alpha = 0.05
if pval > alpha:
    print('same distributions/same group proportion/fail to reject H0: THE EFFECT DOES NOT EXIST IN THE POPULATION')
else: 
    print('different distributions/different group proportions/reject H0: THE EFFECT EXISTS IN THE POPULATION')

0.000
different distributions/different group proportions/reject H0: THE EFFECT EXISTS IN THE POPULATION


## Model 

In [13]:
# Choose features for propensity score calculation
X = df[['tenure','age', 'male','songsListened', 'lovedTracks', 'posts', 'playlists', 'shouts']]
y = df['treatment']

X.head()

,tenure,age,male,songsListened,lovedTracks,posts,playlists,shouts
2,59.0,22.0,0.0,9687,194,0,1,8.0
5,35.0,35.0,0.0,0,0,0,0,0.0
6,42.0,27.0,1.0,508,0,0,1,2.0
9,25.0,21.0,0.0,1357,32,0,0,1.0
11,67.0,24.0,0.0,89984,20,2,0,81.0


In [14]:
#Use logistic regression to calculate the propensity scores

lr = LogisticRegression()
lr.fit(X, y)

/Users/pabloaguirresolana/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [15]:
#get de coefficients
lr.coef_.ravel() 

#get de feature names
X.columns.to_numpy()

#combine features and coefficients into a data frame
coeffs = pd.DataFrame({
    'column':X.columns.to_numpy(),
    'coeff':lr.coef_.ravel(),
})
coeffs

,column,coeff
0,tenure,-0.010246
1,age,-0.051259
2,male,-0.002360
3,songsListened,0.000013
4,lovedTracks,0.001441
5,posts,0.000783
6,playlists,0.000141
7,shouts,0.003560


In [16]:
# get the odds of the model 
odds = np.exp(lr.coef_[0]) 
oddsD = pd.DataFrame(odds, X.columns, columns=['odds']).sort_values(by='odds', ascending=False)
oddsD 

,odds
shouts,1.003566
lovedTracks,1.001442
posts,1.000784
playlists,1.000141
songsListened,1.000013
male,0.997643
tenure,0.989807
age,0.950033


In [17]:
# make predictions 
pred_binary = lr.predict(X)  
pred_prob = lr.predict_proba(X)

In [18]:
# The propensity score (ps) will be the probability of being label 1.
df['ps'] = pred_prob[:, 1]

## Matching

In [20]:
# use 25% of standard deviation of the propensity score as the caliper/radius
# get the k closest neighbors for each observations

caliper = np.std(df.ps) * 0.25
print(f'caliper (radius) is: {caliper:.4f}')

n_neighbors = 10

# setup knn
knn = NearestNeighbors(n_neighbors=n_neighbors, radius=caliper)

ps = df[['ps']]  # double brackets as a dataframe
knn.fit(ps)

# distances and indexes
distances, neighbor_indexes = knn.kneighbors(ps)

print(neighbor_indexes.shape)

# the 10 closest points to the first point
print(distances[0])
print(neighbor_indexes[0])

caliper (radius) is: 0.0382
(43827, 10)
[0.00000000e+00 1.79340463e-06 2.63880097e-06 4.04323260e-06
 1.09131872e-05 1.33684001e-05 1.35625926e-05 1.44923358e-05
 1.93479507e-05 2.14911872e-05]
[    0  7456 15006 38524 28470 14901 18336 22980 16296 13667]


In [21]:
# it is important to reset the index to make the matching, otherwise the matching will be difficult and erratic.
df.reset_index()
df.index = np.arange(1, len(df) + 1)

In [22]:
# for each point in treatment, we find a matching point in control without replacement
# note the 10 neighbors may include both points in treatment and control
matched_control = []  # keep track of the matched observations in control

for current_index, row in df.iterrows():  # iterate over the dataframe
    if row.treatment == 0:  # the current row is in the control group
        df.loc[current_index, 'matched'] = np.nan  # set matched to nan
    else: 
        for idx in neighbor_indexes[current_index, :]: # for each row in treatment, find the k neighbors
            # make sure the current row is not the idx - don't match to itself
            # and the neighbor is in the control 
            if (current_index != idx) and (df.loc[idx].treatment == 0):
                if idx not in matched_control:  # this control has not been matched yet
                    df.loc[current_index, 'matched'] = idx  # record the matching
                    matched_control.append(idx)  # add the matched to the list
                    break


print('total observations in treatment:', len(df[df.treatment==1]))
print('total matched observations in control:', len(matched_control))

total observations in treatment: 9823
total matched observations in control: 9738


In [23]:
# control have no match
treatment_matched = df.dropna(subset=['matched'])  # drop not matched

# matched control observation indexes
control_matched_idx = treatment_matched.matched
control_matched_idx = control_matched_idx.astype(int)  # change to int
control_matched = df.loc[control_matched_idx, :]  # select matched control observations

# combine the matched treatment and control
df_matched = pd.concat([treatment_matched, control_matched])

df_matched.treatment.value_counts()

1    9738
0    9738
Name: treatment, dtype: int64

## Proof of independence (Causality)

If there is independence (p-value > 0.05) between treatment and control group, we can say that there is a casual link between the dependet variable ('adopter') and the treatmente group = 1 (having friends that are subscribers)

In [24]:
# get proportions of dependent variable (adopters) among treament and contro group

pd.crosstab(index = df_matched['adopter'], columns = df_matched['treatment'], margins = True)

treatment,0,1,All
adopter,,,
0,9236,8040,17276
1,502,1698,2200
All,9738,9738,19476


In [25]:
# perform T-test of proportions 
stat, pval = proportions_ztest([502, 1698], [9738, 9738])
print('{0:0.3f}'.format(pval))
alpha = 0.05
if pval > alpha:
    print('same distributions/same group proportion/fail to reject H0: THE EFFECT DOES NOT EXIST IN THE POPULATION')
else: 
    print('different distributions/different group proportions/reject H0: THE EFFECT EXISTS IN THE POPULATION')


0.000
different distributions/different group proportions/reject H0: THE EFFECT EXISTS IN THE POPULATION
